## Stock Market Prediction and Forecasting using Stacked LSTM

### Step 1: Data Collection

In [1]:
import pandas_datareader as pdr

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [ ]:
import pandas as pd
train=pd.read_csv(r"C:\Users\Sourabh\Downloads\^NSEI.csv")
train.head()

In [ ]:
train.tail()

In [ ]:
df1=train.reset_index()['Close']

In [ ]:
df1.shape

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df1)

### -- LSTM are sensitive to the scale of the data, so we apply MinMax Scaler

In [ ]:
import numpy as np 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [ ]:
df1

In [ ]:
df1.shape

## Step 2: Splitting Dataset into Train Split and Test Split

In [ ]:
training_size=int(len(df1)*0.65)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

In [ ]:
training_size,test_size

In [ ]:
train_data

## -- Convert an array of values into a dataset matrix

In [ ]:
import numpy
def create_dataset(dataset,time_step=1):
    dataX,dataY=[],[]
    for i in range(len(dataset)-time_step-1):
        a=dataset[1:(i+time_step),0]          ### i=0; 0,1,2,3,...
        dataX.append(a)
        dataY.append(dataset[i+time_step,0])
        return numpy.array(dataX),numpy.array(dataY)

## -- Reshape into X=t,t+1,t+2,t+3 and y=t+4

In [ ]:
time_step=100
X_train,y_train=create_dataset(train_data,time_step)
X_test,y_test=create_dataset(test_data,time_step)

In [ ]:
print(X_train)

In [ ]:
print(X_train.shape),print(y_train.shape)

In [ ]:
print(X_test.shape),print(y_test.shape)

## -- Reshape input to be [samples, time steps, features] which is required for LSTM

In [ ]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

## Step 3: Create a Stacked LSTM Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(99,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64,verbose=1)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

## -- Implementing Prediction and Checking Performnce Metrics

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

## -- Transform Back to Original Form

In [ ]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

## -- Calculating RMSE Performance Metrics

In [ ]:
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

## -- Test Data RMSE 

In [ ]:
math.sqrt(mean_squared_error(y_test,test_predict))

## Step 3.5: Plotting

In [ ]:
## Shift Train Predictions for Plotting
look_back=99
trainPredictPlot = numpy.empty_like(df1)
trainPredictPlot[:,:] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back,:] = train_predict
## Shift Test Predictions for Plotting
testPredictPlot = numpy.empty_like(df1)
testPredictPlot[:,:] = numpy.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1,:] = test_predict
## Plot Baseline And Predictions
plt.plot(scaler.inverse_transform(df1))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

## Step 4: Prediction for next 30 days and plotting of the output

In [ ]:
len(test_data)

In [ ]:
x_input=test_data[333:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
temp_input

## -- Demonstrating the Prediction for next 30 Days

In [ ]:
from numpy import array
lst_output=[]
n_steps=100
i=0
while(i<30):
    if(len(temp_input)>100):
        ## print(test_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        ## print(x_input)
        x_input=x_input.reshape(1,-1)
        x_input=x_input.reshape(1,n_steps,1)
        yhat=model.predict(x_input,verbose=0)
        print("{} day input {}".format(i,yhat))
        ## print(temp_input)
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input=x_input.reshape(1,n_steps,1)
        yhat=model.predict(x_input,verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
print(lst_output)

## -- Plotting Prediction for next 30 days

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,131)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(df1)

In [ ]:
df3=df1.tolist()
df3.extend(lst_output)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(df1[1135:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
plt.plot(df3[1000:])

In [ ]:
plt.plot(df3[1200:])

## Step : Prediction for next 7 days and plotting of the output

## -- Demonstrating the Prediction for next 7 Days

In [ ]:
from numpy import array
lst_output=[]
n_steps=100
i=0
while(i<7):
    if(len(temp_input)>100):
        ## print(test_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        ## print(x_input)
        x_input=x_input.reshape(1,-1)
        x_input=x_input.reshape(1,n_steps,1)
        yhat=model.predict(x_input,verbose=0)
        print("{} day input {}".format(i,yhat))
        ## print(temp_input)
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input=x_input.reshape(1,n_steps,1)
        yhat=model.predict(x_input,verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
print(lst_output)

## -- Plotting Prediction for next 7 days

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,108)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(df1)

In [ ]:
df3=df1.tolist()
df3.extend(lst_output)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(df1[1135:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
plt.plot(df3[1000:])

In [ ]:
plt.plot(df3[1200:])

## Step 6: Prediction for next 365 days and plotting of the output

## -- Demonstrating the Prediction for next 7 Days

In [ ]:
from numpy import array
lst_output=[]
n_steps=100
i=0
while(i<365):
    if(len(temp_input)>100):
        ## print(test_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        ## print(x_input)
        x_input=x_input.reshape(1,-1)
        x_input=x_input.reshape(1,n_steps,1)
        yhat=model.predict(x_input,verbose=0)
        print("{} day input {}".format(i,yhat))
        ## print(temp_input)
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input=x_input.reshape(1,n_steps,1)
        yhat=model.predict(x_input,verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
print(lst_output)

## -- Plotting Prediction for next 365 days

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,466)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(df1)

In [ ]:
df3=df1.tolist()
df3.extend(lst_output)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(df1[1135:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
plt.plot(df3[1000:])

In [ ]:
plt.plot(df3[1200:])